#**D605 Optimization**  
## Performance Assessment #3 – Solving an Optimization PRoblem

Bernard Connelly  
Master of Science, Data Analytics, Western Governors University  
Dr. Taylor Jensen

---

In [1]:
##!pip install pulp

# Packages and Libraries
import pandas as pd # Calculations
import numpy as np # Dataframes
import warnings # Squelch warnings
warnings.filterwarnings("ignore") # Squelch warnings
import pulp # Defining Variables & Constraints + Solver

#**Part 1: Using Mathematical Computations to Solve the Amazon Distribution Problem**

### A1. Demonstrate that the solver provided a solution to the optimization problem
PuLP is imported to define various components of the optimization model, and implement CBC as a solver to produce a solution. The variables and mathematical formulas which were identified in Performance Assessment #2 will be input into the model before solving. Code utilized to accomplish this is listed below. Notably, all capacity is measured in **monthly tons.**


In [2]:
# Setting up Hubs, Focus Cities, and Centers with Demand

## Hubs
hubs = ['CVG', 'AFW']
hub_capacity = {'CVG': 95650, 'AFW': 44350}

current_tonnage = {'CVG': 82800, 'AFW': 38400}
available_capacity = {
    h: hub_capacity[h] - current_tonnage[h] for h in hubs
}

## Focus Cities
focus_cities = ['Leipzig', 'Hyderabad', 'SanBernardino']
focus_capacity = {'Leipzig': 85000, 'Hyderabad': 19000, 'SanBernardino': 36000}

## Centers
center_demand = {
        "Paris": 6500, "Cologne": 640, "Hanover": 180, "Bangalore": 9100, "Coimbatore": 570,
    "Delhi": 19000, "Mumbai": 14800, "Cagliari": 90, "Catania": 185, "Milan": 800,
    "Rome": 1700, "Katowice": 170, "Barcelona": 2800, "Madrid": 3700, "Castle Donington": 30,
    "London": 6700, "Mobile": 190, "Anchorage": 175, "Fairbanks": 38, "Phoenix": 2400,
    "Los Angeles": 7200, "Ontario": 100, "Riverside": 1200, "Sacramento": 1100,
    "San Francisco": 1900, "Stockton": 240, "Denver": 1500, "Hartford": 540, "Miami": 3400,
    "Lakeland": 185, "Tampa": 1600, "Atlanta": 3000, "Honolulu": 500, "Kahului/Maui": 16,
    "Kona": 63, "Chicago": 5100, "Rockford": 172, "Fort Wayne": 200, "South Bend": 173,
    "Des Moines": 300, "Wichita": 290, "New Orleans": 550, "Baltimore": 1300,
    "Minneapolis": 1700, "Kansas City": 975, "St. Louis": 1200, "Omaha": 480,
    "Manchester": 100, "Albuquerque": 450, "New York": 11200, "Charlotte": 900,
    "Toledo": 290, "Wilmington": 150, "Portland": 1200, "Allentown": 420, "Pittsburgh": 1000,
    "San Juan": 1100, "Nashville": 650, "Austin": 975, "Dallas": 3300, "Houston": 3300,
    "San Antonio": 1100, "Richmond": 600, "Seattle/Tacoma": 2000, "Spokane": 260
}

Defining Network Nodes and Capacity Constraints

The above section establishes the key supply chain components within the Amazon Air logistics network:

- **Hubs:** These are the two main cargo centers (`CVG` and `AFW`) that serve as starting points for shipments. Each hub has a maximum monthly capacity and a current tonnage level already in use. The model calculates the remaining available capacity by subtracting current tonnage from total capacity, ensuring that only unused volume is considered in the optimization.

- **Focus Cities:** These are intermediate air cargo centers that receive shipments from hubs and forward them to distribution centers. Each focus city is assigned a fixed capacity limit, representing its throughput constraints.

- **Distribution Centers:** These are the final destinations that require specific quantities of cargo. The `center_demand` dictionary lists 65 distribution centers along with their respective monthly demand values in tons.

Together, these definitions provide the structural foundation for the optimization model. The hub and focus city capacities serve as supply constraints, while the distribution center demands represent the fulfillment requirements that must be satisfied by the solution.


In [3]:
# Creating Cost Data

INF = 9999 # "Infinite" value to flag routes as invalid

## Cost from Hubs to Focus Cities
cost_hub_to_focus = {
    ('CVG', 'Leipzig'): 1.5,
    ('CVG', 'SanBernardino'): 0.5,
    ('AFW', 'SanBernardino'): 0.5,
}

## Cost from Hubs to Centers
allowed_hub_to_center = {
    ('CVG', 'Paris'), ('CVG', 'Cologne'), ('CVG', 'Hanover'),
    ('CVG', 'Barcelona'), ('CVG', 'Madrid'), ('CVG', 'Castle Donington'),
    ('CVG', 'London'), ('CVG', 'Rome'),
    ('AFW', 'Mobile'), ('AFW', 'Austin'), ('AFW', 'Dallas'),
    ('AFW', 'Houston'), ('AFW', 'San Antonio'),
    ('AFW', 'Phoenix'), ('AFW', 'San Juan'), ('AFW', 'Charlotte'),
}

cost_hub_to_center = {}
for hub in hubs:
    for center in center_demand:
        if (hub, center) in allowed_hub_to_center:
            cost_hub_to_center[(hub, center)] = 0.5
        else:
            cost_hub_to_center[(hub, center)] = INF


# Cost from Focus Cities to Centers (only valid routes)
cost_focus_to_center = {}
valid_costs = {
    ('Leipzig', 'Paris'): 0.5,
    ('Leipzig', 'Cologne'): 0.5,
    ('Leipzig', 'Hanover'): 0.5,
    ('Leipzig', 'Cagliari'): 0.5,
    ('Leipzig', 'Catania'): 0.5,
    ('Leipzig', 'Milan'): 0.5,
    ('Leipzig', 'Rome'): 0.5,
    ('Leipzig', 'Katowice'): 0.5,
    ('Leipzig', 'Barcelona'): 0.5,
    ('Leipzig', 'Madrid'): 0.5,
    ('Leipzig', 'Castle Donington'): 0.5,
    ('Leipzig', 'London'): 0.75,
    ('Leipzig', 'New York'): 1.6,
    ('Leipzig', 'Hartford'): 1.5,
    ('Leipzig', 'Allentown'): 1.5,

    ('Hyderabad', 'Bangalore'): 0.5,
    ('Hyderabad', 'Coimbatore'): 0.5,
    ('Hyderabad', 'Delhi'): 0.5,
    ('Hyderabad', 'Mumbai'): 0.5,
    ('Hyderabad', 'Paris'): 1.1,
    ('Hyderabad', 'Cologne'): 1.0,
    ('Hyderabad', 'Hanover'): 1.0,
    ('Hyderabad', 'Cagliari'): 1.0,
    ('Hyderabad', 'Rome'): 1.1,
    ('Hyderabad', 'Madrid'): 1.1,
    ('Hyderabad', 'London'): 1.1,

    ('SanBernardino', 'Mobile'): 0.5,
    ('SanBernardino', 'Anchorage'): 0.7,
    ('SanBernardino', 'Fairbanks'): 0.7,
    ('SanBernardino', 'Phoenix'): 0.5,
    ('SanBernardino', 'Sacramento'): 0.5,
    ('SanBernardino', 'San Francisco'): 0.5,
    ('SanBernardino', 'Stockton'): 0.5,
    ('SanBernardino', 'Denver'): 0.5,
    ('SanBernardino', 'Hartford'): 0.5,
    ('SanBernardino', 'Miami'): 0.7,
    ('SanBernardino', 'Lakeland'): 0.7,
    ('SanBernardino', 'Tampa'): 0.7,
    ('SanBernardino', 'Atlanta'): 0.6,
    ('SanBernardino', 'Honolulu'): 0.5,
    ('SanBernardino', 'Kahului/Maui'): 0.5,
    ('SanBernardino', 'Kona'): 0.5,
    ('SanBernardino', 'Chicago'): 0.5,
    ('SanBernardino', 'Rockford'): 0.5,
    ('SanBernardino', 'Fort Wayne'): 0.5,
    ('SanBernardino', 'South Bend'): 0.5,
    ('SanBernardino', 'Des Moines'): 0.5,
    ('SanBernardino', 'Wichita'): 0.5,
    ('SanBernardino', 'New Orleans'): 0.5,
    ('SanBernardino', 'Baltimore'): 0.7,
    ('SanBernardino', 'Minneapolis'): 0.5,
    ('SanBernardino', 'Kansas City'): 0.5,
    ('SanBernardino', 'St. Louis'): 0.5,
    ('SanBernardino', 'Omaha'): 0.5,
    ('SanBernardino', 'Manchester'): 0.7,
    ('SanBernardino', 'Albuquerque'): 0.5,
    ('SanBernardino', 'New York'): 0.7,
    ('SanBernardino', 'Charlotte'): 0.7,
    ('SanBernardino', 'Toledo'): 0.5,
    ('SanBernardino', 'Wilmington'): 0.7,
    ('SanBernardino', 'Portland'): 0.5,
    ('SanBernardino', 'Allentown'): 0.7,
    ('SanBernardino', 'Pittsburgh'): 0.6,
    ('SanBernardino', 'San Juan'): 1.0,
    ('SanBernardino', 'Nashville'): 0.5,
    ('SanBernardino', 'Austin'): 0.5,
    ('SanBernardino', 'Dallas'): 0.5,
    ('SanBernardino', 'Houston'): 0.5,
    ('SanBernardino', 'San Antonio'): 0.5,
    ('SanBernardino', 'Richmond'): 0.7,
    ('SanBernardino', 'Seattle/Tacoma'): 0.5,
    ('SanBernardino', 'Spokane'): 0.5,
}

## Create the focus-to-center cost dictionary from valid_costs
cost_focus_to_center = {}
for (fc, center), cost in valid_costs.items():
    cost_focus_to_center[(fc, center)] = cost

## Fill in INF for all other focus-to-center routes
for fc in focus_cities:
    for center in center_demand:
        if (fc, center) not in cost_focus_to_center:
            cost_focus_to_center[(fc, center)] = INF

### Cost Data and Route Validity

The above section defines the transportation cost structure between hubs, focus cities, and distribution centers. The cost values represent the relative cost per ton of shipping cargo between locations.

- The constant `INF` is used to represent an effectively infinite cost for invalid or disallowed routes.
- Only specific hub-to-focus city and hub-to-center connections are permitted, based on operational constraints. All other routes are assigned `INF` to block them from use in the optimization.
- The `valid_costs` dictionary defines the allowed focus-to-center routes, with associated shipping costs. These are manually curated based on regional service feasibility or existing Amazon Air operations.
- The `cost_focus_to_center` dictionary is built from `valid_costs`, and all missing focus-to-center combinations are explicitly filled with `INF`. This ensures the solver only considers valid paths and blocks all invalid ones by design.

By establishing this layered cost structure up front, the model enforces logistical feasibility and operational limits as it seeks the lowest-cost routing solution.


In [4]:
# Initiating Model and Defining Decision Variables
model = pulp.LpProblem("AmazonAir_Optimization", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts("x", [(i, j) for i in hubs for j in focus_cities], lowBound=0)
y = pulp.LpVariable.dicts("y", [(i, k) for i in hubs for k in center_demand], lowBound=0)
z = pulp.LpVariable.dicts("z", [(j, k) for j in focus_cities for k in center_demand], lowBound=0)

### Model Initialization and Decision Variables

This section initiates the linear programming model using the `PuLP` library, specifying that the objective is to **minimize total transportation cost** (`LpMinimize`).

Three sets of decision variables are defined:

- `x[i, j]`: Represents the quantity of cargo (in tons) shipped from hub `i` to focus city `j`.
- `y[i, k]`: Represents the quantity of cargo shipped directly from hub `i` to distribution center `k`.
- `z[j, k]`: Represents the quantity of cargo shipped from focus city `j` to distribution center `k`.

All variables are constrained to be non-negative (`lowBound=0`), reflecting real-world conditions where negative cargo quantities are not possible.

These decision variables form the core of the optimization problem, enabling the model to determine the optimal routing of cargo across the Amazon Air network while respecting cost and capacity constraints.


In [5]:
# Defining Centers, filling invalids with INF, blocking invalid routes

# Block invalid hub-to-center routes
for (i, k), cost in cost_hub_to_center.items():
    if cost == INF:
        model += y[i, k] == 0, f"Block_Hub_{i}_To_Center_{k}"

# Block invalid focus-to-center routes
for (j, k), cost in cost_focus_to_center.items():
    if cost == INF:
        model += z[j, k] == 0, f"Block_Focus_{j}_To_Center_{k}"

### Blocking Invalid Transportation Routes

This section enforces routing restrictions by explicitly prohibiting invalid or infeasible cargo movements.

- **Hub-to-Center Constraints:** For each hub-to-distribution center pair, if the associated cost in the `cost_hub_to_center` dictionary is set to `INF`, the model adds a constraint that forces the shipment quantity (`y[i, k]`) to be exactly zero. This ensures that the solver cannot use any hub-to-center route that is not allowed operationally.

- **Focus-to-Center Constraints:** Similarly, for each focus city–to–distribution center pair, if the cost is `INF` in the `cost_focus_to_center` dictionary, the model blocks that route by adding a constraint setting `z[j, k] = 0`.

By doing this, the model simulates real-world constraints where certain routes do not exist, are too costly, or are otherwise unavailable for use. This step reinforces feasibility and preserves the integrity of the routing logic by eliminating any unintended "back door" solutions the solver might otherwise attempt.


In [6]:
# Defining Decision Variables, Objectives + Constraints

# Objective
model += (
    pulp.lpSum(cost_hub_to_focus.get((i, j), INF) * x[i, j] for i in hubs for j in focus_cities) +
    pulp.lpSum(cost_hub_to_center[i, k] * y[i, k] for i in hubs for k in center_demand) +
    pulp.lpSum(cost_focus_to_center[j, k] * z[j, k] for j in focus_cities for k in center_demand)
), "Total_Cost"

# Constraints
## Hub available capacity (adjusted for current tonnage)
for i in hubs:
    model += (
        pulp.lpSum(x[i, j] for j in focus_cities) +
        pulp.lpSum(y[i, k] for k in center_demand)
        <= available_capacity[i], f"HubCap_{i}"
    )

## Flow balance for each focus city
for j in focus_cities:
    model += (
        pulp.lpSum(x[i, j] for i in hubs) ==
        pulp.lpSum(z[j, k] for k in center_demand), f"FocusBalance_{j}"
    )

## Focus city capacity
for j in focus_cities:
    model += (
        pulp.lpSum(z[j, k] for k in center_demand)
        <= focus_capacity[j], f"FocusCap_{j}"
    )

## Meet each center’s demand
for k in center_demand:
    model += (
        pulp.lpSum(y[i, k] for i in hubs) +
        pulp.lpSum(z[j, k] for j in focus_cities)
        == center_demand[k], f"Demand_{k}"
    )

### Objective Function and Model Constraints

This section defines the **objective** of the model and imposes the core **constraints** that govern feasible cargo flow.

#### Objective Function:
The model minimizes the total cost of transporting cargo across the Amazon Air network. The total cost is composed of:
- Shipments from hubs to focus cities (`x[i, j]`)
- Direct shipments from hubs to distribution centers (`y[i, k]`)
- Shipments from focus cities to distribution centers (`z[j, k]`)

Each term is multiplied by its corresponding route cost, and all costs are summed using `pulp.lpSum`.

#### Constraints:
To ensure the solution adheres to real-world logistics rules, the following constraints are applied:

- **Hub Capacity Constraints:** Each hub can only send out as much cargo as it has remaining capacity (`available_capacity[i]`), considering both direct and indirect shipments.
  
- **Focus City Flow Balance:** The amount of cargo received by each focus city from hubs must exactly match the amount sent out to distribution centers — maintaining a balanced flow.

- **Focus City Capacity Constraints:** Each focus city cannot exceed its specified capacity for handling outbound shipments.

- **Distribution Center Demand Constraints:** Each distribution center must receive exactly the amount of cargo specified in the `center_demand` dictionary, whether from a hub directly or routed through a focus city.

Together, these constraints ensure that all capacity and demand conditions are respected while the model searches for the least-cost solution.


In [7]:
# Running model
model.solve()

-1

In [8]:
# Check to identify which constraints are causing the model to fail
print("Solver Status:", pulp.LpStatus[model.status])
print("Objective Value:", pulp.value(model.objective))

for v in model.variables():
    if v.varValue and v.varValue > 0:
        print(f"{v.name} = {v.varValue:.2f}")


Solver Status: Infeasible
Objective Value: 337996540.0
x_('AFW',_'Hyderabad') = 5950.00
x_('CVG',_'Hyderabad') = 27850.00
x_('CVG',_'SanBernardino') = 11200.00
z_('Hyderabad',_'Delhi') = 19000.00
z_('Hyderabad',_'Mumbai') = 14800.00
z_('SanBernardino',_'New_York') = 11200.00


### Explanation of why the model is infeasible

Multiple iterations of the code resulted in the model repeatedly yielding `-1`. This indicates the model is infeasible in its current form with its capacites. The code blocks above initiate the model, but also identify the connections and constraints that are causing the model to return an infeasible result. Careful review and modifications of the code to include overlooked constraints and incorrectly assigned capacities still result in an infeasible model. After review the infeasible status is due to a real-world mismatch between demand and supply in the Amazon air problem.

#### **Key Constraints that Contribute**
- Total monthly demand across all 65 distribution centers is approximately 167,870 tons
- After accounting for current cargo levels at the hubs:
- CVG available: 95,650 − 82,800 = 12,850 tons
- AFW available: 44,350 − 38,400 = 5,950 tons
- Combined hub capacity: 18,800 tons
- Combined focus city capacity: 140,000 tons
- Total usable capacity: 158,800 tons

Since  167,870 tons > 158,800 tons, the solver is unable to find a solution that satisfies every center’s demand without exceeding the available capacity at hubs or focus cities — making the problem mathematically infeasible.

The model designed in PA2 is unable to function in a real-world scenario.

#**Part 2: Analyzing the Output of the Model**

###B1. Demonstration that Constraints are Satisfied
Despite the model returning an "Infeasible" solver status, the output indicates that the optimization problem is constructed correctly and the constraints are actively being enforced. The infeasibility arises not due to an error in formulation, but due to real-world constraint violations - specifically, excess demand compared to available capacity. As an example,

The constraint on Hyderabad's focus city capacity was pushed past its limit. Although Hyderabad is assigned 33,800 tons (combined 27850 from CVG and 5950 from AFW), its defined maximum was 19,000. Further, the current demand from New York (11,200), Delhi (19,000), and Mumbai (14,800) could not be absorbed through legal routes without exceeding the focus city limits.

This output demonstrates that the constraints, such as focus city capacity, hub throughput, and route availability (via the blocking with INF), are being respected. Since the solver cannot find a way to meet demand without violating these constraints, it returns an infeasible result. This supports the conclusion that the constraints are functioning as intended.



###B2. Demonstration of Decision Variables, Constraints and Objective Function

The decision variables, constraints and objective function identified in task two can be briefly described as follows:

**Decision Variables:**
- x[i,j]: Shipments from hubs to focus cities.
- y[i,k]: Direct shipments from hubs to distribution centers.
- z[j,k]: Shipments from focus cities to distribution centers.

**Constraints:**
- Each hub cannot exceed its remaining capacity.
- Each focus city cannot ship more than its monthly capacity.
- Flow into a focus city must equal flow out.
- Each center’s demand must be exactly met by incoming shipments.
- Invalid routes are blocked using INF and set to zero.

**Objective Function:**  
Minimize the total cost across all valid shipment paths using cost dictionaries and linear sums.

These elements were programmatically constructed and represented in the code above - the output confirms they are in use.

###B3. Explanation of Output vs. Expected Results

Multiple iterations and updates of the code were made to yield a feasible solution for the model. These included artifically reducing the current tonnage of the hubs, as well as increasing capacity of some focus cities. In each iteration, however, the model remained infeasible.

Upon review, the determination was made the issue lied not in the code, but in the structure of the scenario. Even with additional hub capacity, the permitted shipping routes via defined costs and blocked routes restrict delivery. Additionally, specific centers such as Delhi, Mumbai, and New York can **only** be accessed through particular focus cities, whose capacity becomes a secondary bottleneck. As such, the model performs as expected, returning infeasibility where real-world constraints are over-constrained. The output supports the hypothesis that the issue lies in the problem scenario, not the model logic.

#**Part 3: Reflection**

###C: Reflection on the Modeling Process**  
The development of this model highlighted the complexity of translating logistical scenarios into solvable optimization problems. Initially, I assumed the design of the model, weights, costs and capacities would be easily solvable if the data was plugged in properly to Python - it would simply find a way through to solve. As I worked through the problem, however, I understand that the solution may need to exist outside the bounds of the current system offered by Amazon.

Even by modifying capacities and weight values at the various points in the initial setup, the constraints and decision variables provided limited the model's ability to offer a viable solution, and additional bottlenecks appeared. In this situation, it would be prudent to inform the customer, Amazon in this case, that their current model is unsustainable, and additional considerations need to be taken into account, such as expanding the capacities of their hubs and focus cities to match the increasing demand for their services.

Although I followed my Task 2 assumptions faithfully, those assumptions did not account for interdependent constraints becoming over-restrictive. What began as a straightforward transportation problem became a deeper test of feasibility modeling. This process gave me a clearer appreciation for how mathematical soundness does not guarantee real-world feasibility. Most critically, the process taught me to understand that infeasibility is not failure, but can be validation of a working model.

In future optimization projects, I will be more cautious about assuming the input assumptions allow for solvability within practical bounds.

#**Sources**

No other sources were used outside of the WGU course materials provided